In [ ]:
import math
import numpy as np

source_domain_list = ['1', '2', '3', '4', '5', '6', '7', '8', '9', '10',
                      '11', '12', '13', '14', '15', '16', '17', '18', '19', '20',
                      '21', '22', '23', '24', '25', '26', '27', '28', '29', '30']

def get_k_fold_data(k, i, X):
    """
    返回第i折训练的训练集的志愿者编号和测试集志愿者编号,X_train_user为训练数据,X_test_user为验证数据
    k: k折交叉验证------->
    i: 第i折作测试集
    X: source_domain_list
    """
    assert k > 1 and i < k
    fold_size = len(X) // k  # 6 每份的个数:数据总条数/折数（组数）
    X_train = []
    for j in range(k):
        idx = slice(j * fold_size, (j + 1) * fold_size)  #slice(start,end,step)切片函数
        # idx 为每组 valid
        X_part = X[idx]
        if j == i: ###第i折作valid
            X_test = X_part
        elif X_train is None:
            X_train = X_part
        else:
            X_train = np.concatenate([X_train, X_part])

    return X_train, X_test

def k_fold(k, X_train, y_train, num_epochs=3,learning_rate=0.001, weight_decay=0.1, batch_size=5):
    train_loss_sum, valid_loss_sum = 0, 0
    train_acc_sum ,valid_acc_sum = 0,0

    for i in range(k):
        data = get_k_fold_data(k, i, X_train, y_train) # 获取k折交叉验证的训练和验证数据
        # net =  Net()  ### 实例化模型
        # train_ls, valid_ls = train(net, *data, num_epochs, learning_rate,\
        #                            weight_decay, batch_size)
        print('*'*25,'第',i+1,'折','*'*25)
        print('train_loss:%.6f'%train_ls[-1][0],'train_acc:%.4f\n'%valid_ls[-1][1],\
              'valid loss:%.6f'%valid_ls[-1][0],'valid_acc:%.4f'%valid_ls[-1][1])
        # train_loss_sum += train_ls[-1][0]
        # valid_loss_sum += valid_ls[-1][0]
        # train_acc_sum += train_ls[-1][1]
        # valid_acc_sum += valid_ls[-1][1]
    print('#'*10,'最终k折交叉验证结果','#'*10)
    print('train_loss_sum:%.4f'%(train_loss_sum/k),'train_acc_sum:%.4f\n'%(train_acc_sum/k),\
          'valid_loss_sum:%.4f'%(valid_loss_sum/k),'valid_acc_sum:%.4f'%(valid_acc_sum/k))

train_domain, test_domain = get_k_fold_data(30,0,source_domain_list)
print(train_domain)
print(test_domain)

In [ ]:
def get_device_data(fold_iter):
    assert fold_iter < 81
    device = ['insole_head', 'insole_arm_l', 'insole_arm_r', 'insole_wrist_l', 'insole_wrist_r',
                'insole_chest', 'insole_knee_l', 'insole_knee_r', 'insole_pocket']
    train_index = int(fold_iter / 9)
    test_index = fold_iter % 9

    return device[train_index], device[test_index]

train_device, test_device = get_device_data(0)
print(train_device)
print(test_device)

**活动分布以及混淆矩阵**

In [ ]:
lbls = ["Walking", "Stairs-Up", "Stairs-Down", "Sitting", "Standing", "Lying"]
def plot_confusion_matrix(cm):
    fig, ax = plt.subplots(figsize = (12,8))
    ax.grid(False)
    im = ax.imshow(cm, interpolation = 'nearest', cmap = plt.cm.Oranges)
    ax.figure.colorbar(im, ax = ax)
    ax.set_xlabel('Predicted label', fontsize=12)
    ax.set_ylabel('True label', fontsize=12)
    ax.set_xticklabels([''] + lbls)
    ax.set_yticklabels([''] + lbls)
    plt.xticks(rotation=90)
    thresh = cm.max() / 2
    for i in range(cm.shape[0]):
        for j in range(cm.shape[1]):
            ax.text(j, i, int(cm[i,j]), ha = "center", va = "center", color = "white" if cm[i,j]> thresh else "black", fontsize=16)
            fig.tight_layout()

cm = confusion_matrix(Y_test, prediction)
plot_confusion_matrix(cm)

**特征工程**

In [ ]:
import numpy as np
train_x = np.load('/home/intelligence/Robin/Dataset/MM_train_x.npy')
train_x.shape

In [ ]:
mean_feature_dim = np.array([])
std_feature_dim = np.array([])
range_feature_dim = np.array([])
var_feature_dim = np.array([])
for i in range(len(train_x)):
    mean_win = np.mean(train_x[i], axis=0).reshape(1,-1)
    std_win = np.std(train_x[i], axis=0).reshape(1,-1)
    variance = np.var(train_x[i], axis=0).reshape(1,-1)

    data_max = np.max(train_x[i], axis=0).reshape(1,-1)
    data_min = np.min(train_x[i], axis=0).reshape(1,-1)
    range_win = (data_max - data_min)

    mean_feature_dim = np.concatenate((mean_feature_dim, mean_win), axis=0) if mean_feature_dim.size else mean_win  # 平均值
    std_feature_dim = np.concatenate((std_feature_dim, std_win), axis=0) if std_feature_dim.size else std_win  # 标准差
    range_feature_dim = np.concatenate((range_feature_dim, range_win), axis=0) if range_feature_dim.size else range_win  # 范围
    var_feature_dim = np.concatenate((var_feature_dim, variance), axis=0) if var_feature_dim.size else variance  # 方差

In [ ]:
from scipy import stats
# 提取偏度和峰度
Skewness_feature_dim = np.array([])
kurtosis_feature_dim = np.array([])
for i in range(len(train_x)):
    Skewness = stats.skew(train_x[i]).reshape(1,-1)
    Kurtosis = stats.kurtosis(train_x[i]).reshape(1,-1)
    Skewness_feature_dim = np.concatenate((Skewness_feature_dim, Skewness), axis=0) if Skewness_feature_dim.size else Skewness
    kurtosis_feature_dim = np.concatenate((kurtosis_feature_dim, Kurtosis), axis=0) if kurtosis_feature_dim.size else Kurtosis

**TRASEND论文中有进行傅里叶变换**

In [ ]:
# 压力时间积分
# PTI_feature_dim = np.array([])
#
# for i in range(len(train_x)):
#     win_sum = np.zeros(len(train_x[0][0]))
#     for j in range(len(train_x[0])):
#         PTI = train_x[i][j] * (j/350)
#         win_sum += PTI
#     win_sum = win_sum.reshape(1,-1)
#     PTI_feature_dim = np.concatenate((PTI_feature_dim, win_sum), axis=0) if PTI_feature_dim.size else win_sum

In [ ]:
# 提取频域energe特征
# DFR_feature_dim = np.array([])
# for i in range(len(train_x)):
#     data_fft  = np.fft.fft(train_x[i], axis=0)
#     m, N = data_fft.shape
#     mag = np.abs(data_fft)
#     DFR = np.sum((mag[:m, :] ** 2)/m, axis=0).reshape(1,-1)
#     DFR_feature_dim = np.concatenate((DFR_feature_dim, DFR), axis=0) if DFR_feature_dim.size else DFR

In [ ]:
# 提取entropy


In [ ]:
feature_train_x = np.hstack((mean_feature_dim, std_feature_dim, var_feature_dim, range_feature_dim, Skewness_feature_dim, kurtosis_feature_dim))
np.save('/home/intelligence/Robin/Dataset/feature_MM_train_x.npy', feature_train_x)

**特征选择**

In [ ]:
import pandas as pd
from sklearn import metrics
from sklearn.model_selection import train_test_split
import xgboost as xgb
import matplotlib.pyplot as plt

# 导入数据集
df = pd.read_csv("/home/intelligence/Robin/Dataset/diabetes.csv")
data=df.iloc[:,:8]
target=df.iloc[:,-1]

# 切分训练集和测试集
train_x, test_x, train_y, test_y = train_test_split(data,target,test_size=0.2,random_state=7)

# xgboost模型初始化设置
dtrain=xgb.DMatrix(train_x,label=train_y)
dtest=xgb.DMatrix(test_x)
watchlist = [(dtrain,'train')]

# booster:
params={'booster':'gbtree',
        'objective': 'binary:logistic',
        'eval_metric': 'auc',
        'max_depth':5,
        'lambda':10,
        'subsample':0.75,
        'colsample_bytree':0.75,
        'min_child_weight':2,
        'eta': 0.025,
        'seed':0,
        'nthread':8,
        'gamma':0.15,
        'learning_rate' : 0.01}

# 建模与预测：50棵树
bst=xgb.train(params,dtrain,num_boost_round=50,evals=watchlist)
ypred=bst.predict(dtest)

# 设置阈值、评价指标
y_pred = (ypred >= 0.5)*1
print ('Precesion: %.4f' %metrics.precision_score(test_y,y_pred))
print ('Recall: %.4f' % metrics.recall_score(test_y,y_pred))
print ('F1-score: %.4f' %metrics.f1_score(test_y,y_pred))
print ('Accuracy: %.4f' % metrics.accuracy_score(test_y,y_pred))
print ('AUC: %.4f' % metrics.roc_auc_score(test_y,ypred))

ypred = bst.predict(dtest)
print("测试集每个样本的得分\n",ypred)
ypred_leaf = bst.predict(dtest, pred_leaf=True)
print("测试集每棵树所属的节点数\n",ypred_leaf)
ypred_contribs = bst.predict(dtest, pred_contribs=True)
print("特征的重要性\n",ypred_contribs )

xgb.plot_importance(bst,height=0.8,title='影响糖尿病的重要特征', ylabel='特征')
plt.rc('font', family='Arial Unicode MS', size=14)
plt.show()

**DeepSense**

In [3]:
import torch
import math
import torch.nn as nn
import torch.nn.functional as F


class BasicConv(nn.Module):
    def __init__(self, in_planes, out_planes, kernel_size, stride=1, padding=(1, 0), dilation=1, groups=1, relu=True, bn=True, bias=False):
        super(BasicConv, self).__init__()
        self.out_channels = out_planes
        self.conv = nn.Conv2d(in_planes, out_planes, kernel_size=kernel_size, stride=stride, padding=padding, dilation=dilation, groups=groups, bias=bias)
        self.bn = nn.BatchNorm2d(out_planes) if bn else None
        self.relu = nn.ReLU() if relu else None

    def forward(self, x):
        x = self.conv(x)
        if self.bn is not None:
            x = self.bn(x)
        if self.relu is not None:
            x = self.relu(x)
        return x


class ZPool(nn.Module):
    def forward(self, x):
        return torch.cat((torch.max(x, 1)[0].unsqueeze(1), torch.mean(x, 1).unsqueeze(1)), dim=1)


class AttentionGate(nn.Module):
    def __init__(self, temperature):
        super(AttentionGate, self).__init__()
        kernel_size = (5, 1)
        self.temperature = temperature
        self.compress = ZPool()
        # self.conv = BasicConv(2, 1, kernel_size, stride=1, padding=(kernel_size-1) // 2, relu=False)
        self.conv = BasicConv(2, 1, kernel_size, stride=1, padding=(2, 0), relu=False)

    def updata_temperature(self):
        if self.temperature != 1:
            self.temperature -= 3
            print('Change temperature to:', str(self.temperature))

    def forward(self, x):
        # print(x.shape, 'ty1')
        x_compress = self.compress(x)
        # print(x_compress.shape, 'Z_pooling')
        x_out = self.conv(x_compress)
        # print(x_out.shape, 'Conv+BN+RelU')
        # scale = torch.softmax(x_out/self.temperature, 1)
        scale = torch.sigmoid(x_out)
        # print((x*scale).shape, 'ty4')
        return x * scale


class TripletAttention(nn.Module):
    def __init__(self, no_spatial=False, temperature=34):
        super(TripletAttention, self).__init__()

        self.cw = AttentionGate(temperature)
        self.hc = AttentionGate(temperature)
        self.no_spatial = no_spatial

        self.w1 = torch.nn.Parameter(torch.FloatTensor(1), requires_grad=True)
        self.w2 = torch.nn.Parameter(torch.FloatTensor(1), requires_grad=True)
        self.w3 = torch.nn.Parameter(torch.FloatTensor(1), requires_grad=True)

        # initialization
        # self.w1 = torch.nn.init.normal_(self.w1)
        # self.w2 = torch.nn.init.normal_(self.w2)
        # self.w3 = torch.nn.init.normal_(self.w3)
        self.w1.data.fill_(1/3)
        self.w2.data.fill_(1/3)
        self.w3.data.fill_(1/3)

        if not no_spatial:
            self.hw = AttentionGate(temperature)

    def update_temperature(self):
        self.cw.updata_temperature()
        self.hc.updata_temperature()
        self.hw.updata_temperature()

    def forward(self, x):
        x = x.unsqueeze(1)
        x_perm1 = x.permute(0, 2, 1, 3).contiguous()
        x_out1 = self.cw(x_perm1)
        # print(x_out1.shape, 'ty44')
        x_out11 = x_out1.permute(0, 2, 1, 3).contiguous()
        x_perm2 = x.permute(0, 3, 2, 1).contiguous()
        x_out2 = self.hc(x_perm2)
        x_out21 = x_out2.permute(0, 3, 2, 1).contiguous()
        if not self.no_spatial:
            x_out = self.hw(x)
            # print(x_out.shape, 'ty55')
            # x_out = x_out11
            # x_out = 1/3 * (x_out + x_out11 + x_out21)
            # x_out = 4 * x_out + 5 * x_out11 + 6 * x_out21
            x_out = self.w1 * x_out + self.w2 * x_out11 + self.w3 * x_out21
            # print(self.w1, self.w2, self.w3, 'w1,w2,w3')
            # print(x_out.shape, 'ty22')
        else:
            x_out = self.w1 * x_out11 + self.w2 * x_out21
        # return x_out, self.w1, self.w2, self.w3
        return x_out


In [4]:
x = torch.randn(128, 300, 25)
model = TripletAttention()
x = x.to('cuda:0')
model.to('cuda')
out = model(x)
# print(model)
print(out.shape)

torch.Size([128, 1, 300, 25])
